In [ ]:
"""Provenance table"""
from __future__ import absolute_import
import os

try:
    this_dir = os.path.dirname(os.path.abspath(__file__)) + '/'
except :
    this_dir = ''

def provenance_link(uuid, label=None):
    """Return representation of provenance link."""
    import os
    
    if label is None:
        label = "Browse provenance\n" + str(uuid)
        
    logo_url = "select-figure/static/images/ceede967.aiida-logo-128.png"
    #logo_url = "statichttps://www.materialscloud.org/discover/images/ceede967.aiida-logo-128.png"
    explore_url = os.getenv('EXPLORE_URL', "https://dev-www.materialscloud.org/explore/curated-cofs")
    return "<a href='{url}/details/{uuid}' target='_blank'><img src={logo_url} title='{label}' style='width: 20px;  height: auto;'></a>"\
          .format(url=explore_url, uuid=str(uuid), label=label, logo_url=logo_url)

def detail_link(label):
    """Return representation of provenance link."""
    logo_url = 'select-figure/static/images/co2-128.png'
    return "<a href='detail?id={l}' target='_blank'><img src='{u}' style='width: 20px;  height: auto;'></a>".format(l=label,u=logo_url)
    

def get_data_aiida(has_pe=False):
    """Query the AiiDA database"""
    from aiida import load_dbenv, is_dbenv_loaded
    from aiida.backends import settings
    if not is_dbenv_loaded():
        load_dbenv(profile=settings.AIIDADB_PROFILE)
    from aiida.orm.querybuilder import QueryBuilder
    from aiida.orm import WorkCalculation, Node
 
    qb = QueryBuilder()
    qb.append(WorkCalculation, filters={ 'attributes.function_name': {'==': 'collect_outputs'} }, tag='collect')
    qb.append(Node, project=['label', 'uuid'], edge_filters={'label': 'ref_structure'}, input_of='collect')
    qb.append(Node, project=['uuid'], edge_filters={'label': 'opt_cif_ddec'}, input_of='collect')
    if has_pe:
        qb.append(Node, project=['uuid'], edge_filters={'label': 'opt_out_pe'}, input_of='collect')

    return qb.all()


def get_table(): 
    import pandas
    pandas.set_option('max_colwidth', 1000)

    # label, ref_uuid, opt_uuid 
    uuids = get_data_aiida()
    uuid_dict = { v[0]: v[1:] for v in uuids }

    # label, ref_uuid, opt_uuid, pe_uuid
    uuids_pe = get_data_aiida(has_pe=True)
    uuid_dict_pe = { v[0]: v[1:] for v in uuids_pe }


    df = pandas.read_csv(this_dir + "static/cof-frameworks.csv")
    ref_s = []
    opt_s = []
    detail = []
    for label in df['CURATED-COFs ID']:
        try:
            ref, opt = uuid_dict[label]
            ref_s += [provenance_link(ref)]
            opt_s += [provenance_link(opt)]
            
        except KeyError:
            ref_s += ["N/A"]
            opt_s += ["N/A"]

        # Check whether there is an entry in uuid_dict_pe
        # if yes, add detail link
        try:
            uuid_dict_pe[label]
            detail += [detail_link(label)]
            
        except KeyError:
            detail += ["N/A"]
            

    df['Original Structure'] = pandas.Series(ref_s, index=df.index)
    df['Optimized Structure'] = pandas.Series(opt_s, index=df.index)
    df['CCS Workflow'] = pandas.Series(detail, index=df.index)
    return df[ ['CURATED-COFs ID', 'Name', 'Original Structure', 'Optimized Structure', 'CCS Workflow' ]]



In [ ]:
"""Buttons"""
import panel as pn
def fake_button(link, label):
    return """<span><a href="{link}" target="_blank">
        <button class="bk bk-btn bk-btn-primary" type="button">{label}</button></a></span>""".format(link=link,label=label)

buttons = pn.Row()
buttons.append(fake_button(link="https://github.com/danieleongari/CURATED-COFs", label="GitHub repository"))
buttons.append(fake_button(link="https://archive.materialscloud.org/file/2019.0034/v1/cifs_cellopt.zip", label="Optimized Structures (DDEC)"))

In [ ]:
import panel as pn
from panel.interact import interact

pn.extension()

t = pn.Column()
t.append(buttons)
t.append(get_table().to_html(escape=False))
t.servable()